# Cluster-based Input Weight Initialization for Echo State Networks

This notebook aims to be the supplemental material for the corresponding journal article.

We aim to pre-train the input weight matrix of ESNs using the K-Means algorithm since passing features to the non-linear reservoir of ESNs is closely related to compute the dot product between two vectors.

We use various datasets from https://github.com/FilippoMB/Time-series-classification-and-clustering-with-Reservoir-Computing

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

import time
import os
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.base import clone
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, ParameterGrid, cross_val_score
from sklearn.utils import shuffle
from sklearn.utils.fixes import loguniform
from scipy.stats import uniform
from sklearn.cluster import MiniBatchKMeans
from joblib import dump, load
from pyrcn.echo_state_network import ESNClassifier
from pyrcn.base.blocks import PredefinedWeightsInputToNode, NodeToNode
from pyrcn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error
from pyrcn.model_selection import SequentialSearchCV
import matplotlib
import seaborn as sns
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
%matplotlib inline
#Options
plt.rc('image', cmap='RdBu')
plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('axes', labelsize=8)

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker

## Select a dataset

In this study, we have worked with the [datasets](https://mega.nz/#!aZkBwYDa!JZb99GQoUn4EoJYceCK3Ihe04hhYZYuIWn018gcQM8k)[^1]. 

For more information, we refer to the websites of the distinct datasets:
- [Spoken Arabic Digits](https://archive.ics.uci.edu/ml/datasets/Spoken+Arabic+Digit)
- [Australian Sign Language signs (High Quality) Data Set](https://archive.ics.uci.edu/ml/datasets/Australian+Sign+Language+signs+(High+Quality))
- [Character Trajectories Data Set](http://archive.ics.uci.edu/ml/datasets/Character+Trajectories)
- [ChlorineConcentration: Chlorine concentration data set](https://rdrr.io/github/moviedo5/fda.tsc/man/ChlorineConcentration.html)
- [CMU Graphics Lab Motion Capture Database](http://mocap.cs.cmu.edu/)
- ECG Dataset
- Japanese Vowels Data Set
- [Kicks vs. Punch Dataset](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7813879/)
- [Libras Movement Data Set](https://archive.ics.uci.edu/ml/datasets/Libras+Movement)
- NetFlow dataset
- PEMS-SF Data Set
- DistalPhalanxTW Data Set
- [Robot Execution Failures Data Set](http://archive.ics.uci.edu/ml/datasets/Robot+Execution+Failures)
- [SwedishLeaf Data Set](http://www.timeseriesclassification.com/description.php?Dataset=SwedishLeaf)
- [uWave Data Set](https://www.yecl.org/publications/liu09percom.pdf)
- [Wafer Data Set](http://www.timeseriesclassification.com/description.php?Dataset=Wafer)
- [BasicMotions](http://www.timeseriesclassification.com/description.php?Dataset=BasicMotions)

[^1]: https://github.com/FilippoMB/Time-series-classification-and-clustering-with-Reservoir-Computing

In [ ]:
filelist = list(Path(r"E:\multivariate_time_series_dataset\numpy").rglob("*.npz"))
fileMenu = widgets.Dropdown(options=filelist, value=None, description='Choose Dataset:')

In [ ]:
def load_and_preprocess_dataset(fname):
    if fname is not None:
        dataset = np.load(str(fname))

        X_train = np.empty(shape=(dataset['X'].shape[0], ), dtype=object)
        y_train = np.empty(shape=(dataset['X'].shape[0], ), dtype=object)
        X_test = np.empty(shape=(dataset['Xte'].shape[0], ), dtype=object)
        y_test = np.empty(shape=(dataset['Xte'].shape[0], ), dtype=object)
        for k, (X, y) in enumerate(zip(dataset['X'], dataset['Y'])):
            X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
            y_train[k] = np.argwhere(y).ravel()
        scaler = StandardScaler().fit(np.concatenate(X_train))
        for k, X in enumerate(X_train):
            X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        X_train, y_train = shuffle(X_train, y_train, random_state=0)
        for k, (X, y) in enumerate(zip(dataset['Xte'], dataset['Yte'])):
            X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
            y_test[k] = np.argwhere(y).ravel()
        return X_train, X_test, y_train, y_test
    return None

data_widget = interactive(load_and_preprocess_dataset, {"manual": True}, fname=fileMenu)
data_widget

In [ ]:
X_train, X_test, y_train, y_test = data_widget.result

## Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'input_activation': 'identity',
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'reservoir_activation': 'tanh',
                          'leakage': 0.1,
                          'bidirectional': False,
                          'k_rec': 10,
                          'alpha': 1e-3,
                          'random_state': 42,
                          'decision_strategy': "winner_takes_all"}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = ESNClassifier(**initially_fixed_params)
try:
    sequential_search = load("../multidataset/sequential_search_" + os.path.splitext(os.path.basename(fileMenu.label))[0] + ".joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_" + os.path.splitext(os.path.basename(fileMenu.label))[0] + ".joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = clone(sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

## Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'input_activation': 'identity',
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'reservoir_activation': 'tanh',
                          'leakage': 0.1,
                          'bidirectional': False,
                          'k_rec': 10,
                          'alpha': 1e-3,
                          'random_state': 42,
                          'decision_strategy': "winner_takes_all"}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = ESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                            **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_" + os.path.splitext(os.path.basename(fileMenu.label))[0] + "_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_" + os.path.splitext(os.path.basename(fileMenu.label))[0] + "_km.joblib")

In [ ]:
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = ESNClassifier(**constant_params)

kmeans = MiniBatchKMeans(n_clusters=200, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
base_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
base_esn.set_params(**{"hidden_layer_size": 200}, **constant_params)
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = ESNClassifier(**constant_params)
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

In [ ]:
sequential_search = load("../multidataset/sequential_search_chlo_km.joblib")

In [ ]:
# idx = np.random.randint(0, 800, 50)
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.histplot(data=w_in, stat="count", legend=False)
plt.xlabel("Weight")
plt.ylabel("Count")
plt.imshow(sequential_search.best_estimator_.input_to_node.input_weights.todense()[:, idx])
plt.colorbar()
# plt.savefig('KM_ESN_Input_Weight_Hist_CHLO.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step1"])

fig = plt.figure()
ax = sns.scatterplot(x="param_spectral_radius", y="param_input_scaling", hue="mean_test_score", palette='RdBu', data=df)
plt.xlabel("Spectral Radius")
plt.ylabel("Input Scaling")

norm = plt.Normalize(0.97, 1.0)
sm = plt.cm.ScalarMappable(cmap="RdBu", norm=norm)
sm.set_array([])
plt.xlim((0, 2.05))
plt.ylim((0, 1.05))

# Remove the legend and add a colorbar
ax.get_legend().remove()
ax.figure.colorbar(sm)
fig.set_size_inches(4, 2.5)
tick_locator = ticker.MaxNLocator(5)
ax.yaxis.set_major_locator(tick_locator)
ax.xaxis.set_major_locator(tick_locator)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step2"])
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_leakage", y="mean_test_score")
ax.set_xscale('log')
plt.xlabel("Leakage")
plt.ylabel("Score")
plt.xlim((1e-5, 1e0))
tick_locator = ticker.MaxNLocator(10)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.4f'))
plt.grid()

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step3"])
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_bias_scaling", y="mean_test_score")
plt.xlabel("Bias Scaling")
plt.ylabel("Score")
plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
plt.grid()

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step4"])
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_alpha", y="mean_test_score")
ax.set_xscale('log')
plt.xlabel("Alpha")
plt.ylabel("Score")
plt.xlim((1e-5, 1e0))
tick_locator = ticker.MaxNLocator(20)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
plt.grid()